<a href="https://colab.research.google.com/github/Kealfeyne/Samsung_Bootcamp_Classic_ML/blob/main/2_broken_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

В датасете есть лазейка, благодаря которой можно добиться **0.96 accuracy**. Если дата прибытия позже даты смены статуса бронирования, то бронирование проставлялось "отмененным", иначе - "не отмененным". Когда даты совпадают, однозначно статус бронирования сказать не получается.

In [125]:
train = pd.read_csv("train.csv")

In [126]:
train.shape

(44638, 29)

In [127]:
train[['arrival_date_year',	'arrival_date_month', 'arrival_date_day_of_month', 'reservation_status_date', 'is_canceled']].head()

,arrival_date_year,arrival_date_month,arrival_date_day_of_month,reservation_status_date,is_canceled
0,2016,February,12,2016-02-14,0
1,2016,November,18,2016-10-09,1
2,2015,September,7,2015-09-13,0
3,2015,December,6,2015-12-08,0
4,2016,September,29,2016-01-18,1


In [128]:
from datetime import datetime
from tqdm import tqdm

In [152]:
def create_date(dataset):
  """Создает фичу arrival_date на основе arrival_date_year, arrival_date_month, arrival_date_day_of_month"""
  new_dataset = dataset
  ds = [] 
  for i in tqdm(range(dataset.shape[0])):
    month = datetime.strptime(dataset.iloc[i]['arrival_date_month'], '%B').strftime('%m')
    date = str(dataset.iloc[i]['arrival_date_year']) + '-' + str(month) + '-' + str(dataset.iloc[i]['arrival_date_day_of_month'])
    ds.append(date)
  new_dataset['arrival_date'] = ds
  new_dataset['arrival_date'] = pd.to_datetime(new_dataset['arrival_date'])
  new_dataset['reservation_status_date'] = pd.to_datetime(new_dataset['reservation_status_date'])
  return new_dataset

In [130]:
train = create_date(train)

100%|██████████| 44638/44638 [00:24<00:00, 1801.76it/s]


In [131]:
train.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date
0,Resort Hotel,0,3,2016,February,7,12,0,2,2,...,F,0,No Deposit,0,Transient,81.0,1,0,2016-02-14,2016-02-12
1,City Hotel,1,50,2016,November,47,18,1,2,2,...,A,0,No Deposit,0,Transient,93.6,0,2,2016-10-09,2016-11-18
2,City Hotel,0,14,2015,September,37,7,1,5,2,...,F,0,No Deposit,0,Contract,166.0,0,3,2015-09-13,2015-09-07
3,City Hotel,0,18,2015,December,50,6,2,0,2,...,A,0,No Deposit,0,Transient,107.0,0,0,2015-12-08,2015-12-06
4,City Hotel,1,263,2016,September,40,29,2,4,2,...,A,0,No Deposit,0,Transient,100.3,0,0,2016-01-18,2016-09-29


In [132]:
train = train[['arrival_date', 'reservation_status_date', 'is_canceled']]

In [133]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44638 entries, 0 to 44637
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   arrival_date             44638 non-null  datetime64[ns]
 1   reservation_status_date  44638 non-null  datetime64[ns]
 2   is_canceled              44638 non-null  int64         
dtypes: datetime64[ns](2), int64(1)
memory usage: 1.0 MB


In [134]:
train.head()

,arrival_date,reservation_status_date,is_canceled
0,2016-02-12,2016-02-14,0
1,2016-11-18,2016-10-09,1
2,2015-09-07,2015-09-13,0
3,2015-12-06,2015-12-08,0
4,2016-09-29,2016-01-18,1


Подтвердим лазейку следующими вычислениями

In [135]:
#Отношение тех, у кого дата приезда позже даты смена статуса и бронирование отменено, ко всем, у кого дата позже
len(train[(train['arrival_date'] > train['reservation_status_date']) & (train['is_canceled'] == 1)]) / len(train[train['arrival_date'] > train['reservation_status_date']])

1.0

In [136]:
#Отношение тех, у кого дата приезда раньше даты смена статуса и бронирование не отменено, ко всем, у кого дата раньше
len(train[(train['arrival_date'] < train['reservation_status_date']) & (train['is_canceled'] == 0)]) / len(train[train['arrival_date'] < train['reservation_status_date']])

1.0

In [137]:
#Отношение тех, у кого даты совпадают и статус бронирования - "не отменено", ко всем, у кого даты совпадают
len(train[(train['arrival_date'] == train['reservation_status_date']) & (train['is_canceled'] == 0)]) / len(train[train['arrival_date'] == train['reservation_status_date']])

0.32845894263217096

In [139]:
test = pd.read_csv("test.csv")

In [140]:
test = create_date(test)

100%|██████████| 23525/23525 [00:12<00:00, 1903.83it/s]


In [141]:
test.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date
0,City Hotel,73,2016,July,28,6,0,2,1,0.0,...,A,0,No Deposit,0,Transient,107.10,0,0,2016-06-12,2016-07-06
1,City Hotel,37,2015,October,43,24,2,5,1,1.0,...,A,0,No Deposit,0,Transient,87.78,0,0,2015-10-31,2015-10-24
2,City Hotel,190,2017,April,14,6,2,3,2,0.0,...,A,0,No Deposit,0,Transient,88.40,0,0,2016-09-29,2017-04-06
3,City Hotel,287,2016,August,35,24,1,4,2,0.0,...,B,1,No Deposit,0,Transient,76.71,0,0,2016-08-29,2016-08-24
4,Resort Hotel,386,2016,October,43,20,1,3,2,0.0,...,A,0,No Deposit,0,Transient-Party,49.00,0,0,2016-09-14,2016-10-20


In [142]:
test = test[['arrival_date', 'reservation_status_date']]

In [143]:
test

,arrival_date,reservation_status_date
0,2016-07-06,2016-06-12
1,2015-10-24,2015-10-31
2,2017-04-06,2016-09-29
3,2016-08-24,2016-08-29
4,2016-10-20,2016-09-14
...,...,...
23520,2016-05-23,2016-05-29
23521,2016-11-01,2016-11-04
23522,2016-05-16,2016-05-18
23523,2017-03-05,2017-01-11


In [144]:
result = pd.DataFrame([0]*test.shape[0], columns=['is_canceled']).reset_index()

In [145]:
result['is_canceled'] = (test['arrival_date'] > test['reservation_status_date']).astype(int)

In [148]:
result[result['is_canceled'] == False]

,index,is_canceled
1,1,0
3,3,0
7,7,0
11,11,0
17,17,0
...,...,...
23518,23518,0
23520,23520,0
23521,23521,0
23522,23522,0


In [149]:
result.to_csv(f"result.csv", index=False)
files.download(f"result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>